In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import pandas as pd
import os
import random
import numpy as np                               # loading ll necessary libraries 
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, random_split, Dataset
import torch.optim as optim
from torch.autograd import Variable
import torchvision

In [ ]:
os.chdir("/content/sample_data")

In [ ]:
# loading datasets
df_train   = pd.read_csv("train_data.csv")
df_test = pd.read_csv("test_data.csv")
#df_test 


In [ ]:
x_users_train = df_train['user_id'].values
x_items_train = df_train['item_id'].values
users = torch.tensor(x_users_train,dtype=torch.float32)
items = torch.tensor(x_items_train,dtype=torch.float32)


In [ ]:
users = list(df_train.user_id.unique())              # get a unique list of users and items 
items = list(df_train.item_id.unique())
no_users = len(users)
no_items = len(items)
print(no_users)
print(no_items)

2239
2174


In [ ]:
class nucf_dataset(Dataset):
    def __init__(self,filename):
        file_data = pd.read_csv("train_data.csv")
        x_users_train = file_data['user_id'].values
        x_items_train = file_data['item_id'].values
        y_train = file_data['rating'].values
 
        self.users = torch.tensor(x_users_train,dtype=torch.float32)
        self.items = torch.tensor(x_items_train,dtype=torch.float32)
        self.y = y_train

    def __getitem__(self,idx):
        return (self.users[idx],self.items[idx]),self.y[idx]

In [ ]:
 #Contruct a sparse matrix for our users and items containing number of plays
user_sparse_data = sparse.csr_matrix((df_train['rating'],(df_train['user_id'],df_train['item_id'])))
item_sparse_data = sparse.csr_matrix((df_train['rating'],(df_train['item_id'],df_train['user_id'])))

In [ ]:
#embeddding dimensions 
embed_dim = 32
batch_size = 100 
neurons_mlp = []
in_size_mlp = 2*embed_dim
neurons_mlp.append(in_size_mlp)
output_size = 32
neurons_mlp.append(output_size)

drop_out_rate = [0.0,0.1,0.2,0.3]

In [ ]:
class NueMf(nn.Module):
    def __init__(self,no_users,no_items , mlp_layers,embed_dim,neurons):
        super(NueMf, self).__init__()
        input_size = neurons[0]
        output_size = neurons[1]
        #mf part
        self.embedding_user_mf = nn.Embedding(no_users, embed_dim)  #embeddding users and items 

        self.embedding_item_mf = nn.Embedding(no_items, embed_dim)
        
        #mlp part
        self.embedding_user_mlp = nn.Embedding(no_users, embed_dim)  #embeddding users and items 

        self.embedding_item_mlp = nn.Embedding(no_items, embed_dim)




        self.mlp_layers = nn.Sequential()
        for i in range(mlp_layers):
            if i == 0:
                self.mlp_layers.add_module("linear_layer",nn.Linear(input_size, output_size))
                self.mlp_layers.add_module("relu_layer",nn.ReLU())
                self.mlp_layers.add_module("drop_out",nn.Dropout(drop_out_rate[i]))
            else:
                self.mlp_layers.add_module("linear_layer",nn.Linear(output_size, output_size))
                self.mlp_layers.add_module("relu_layer",nn.ReLU())
                self.mlp_layers.add_module("drop_out",nn.Dropout(drop_out_rate[i]))

        self.output = nn.Linear(output_size + embed_dim, 1)


            

        
    def forward(self,users,items):
        mf_user_emb = self.embedding_user_mf(users)
        mf_item_emb = self.embedding_item_mf(items)

        mlp_user_emb = self.embedding_user_mlp(users)
        mlp_item_emb = self.embedding_item_mlp(items)

        mf_emb_vector = mf_user_emb*mf_item_emb
        mlp_emb_vector = torch.cat([mlp_user_emb,mlp_item_emb], dim=1)
        mlp_emb_vector = self.mlp_layers(mlp_emb_vector)
        

        emb_vector = torch.cat([mf_emb_vector,mlp_emb_vector], dim=1)
        preds = torch.sigmoid(self.out(emb_vector))

        return preds
        

model = NueMf(no_users,no_items,3,embed_dim,neurons_mlp)
    